<a href="https://colab.research.google.com/github/Nick088Official/Real-ESRGAN_Pytorch/blob/main/Real_ESRGAN_Pytorch_Inference_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-ESRGAN Pytorch Inference WEB UI

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2107.10833)
[![GitHub Stars](https://img.shields.io/github/stars/Nick088Official/Real-ESRGAN_Pytorch?style=social)](https://github.com/Nick088Official/Real-ESRGAN_Pytorch)

This is a **Practical Image Restoration Demo** of the Pytorch ai-forever custom pretrained Real-ESRGAN models.
We extend the powerful ESRGAN to a practical restoration application (namely, Real-ESRGAN), which is trained with pure synthetic data. <br>
The following figure shows some real-life examples.

Low quality image:

![](https://github.com/Nick088Official/Real-ESRGAN-Pytorch/blob/main/inputs/lr_lion.png?raw=1)

Real-ESRGAN_Pytorch result:

![](https://github.com/Nick088Official/Real-ESRGAN-Pytorch/blob/main/results/sr_lion.png?raw=1)

**Note that Real-ESRGAN may still fail in some cases as the real-world degradations are really too complex.**<br>
Moreover, it **may not** perform well on human faces, you could try [GFPGAN](https://colab.research.google.com/github/Nick088Official/GFPGAN-Fix/blob/master/GFPGAN_fix_inference.ipynb) for that
<br>

This is the Pytorch Implementation of https://github.com/ai-forever/Real-ESRGAN

Google Colab Notebook based on [the Hugging Face Space](https://huggingface.co/spaces/Nick088/Real-ESRGAN_Pytorch)

**Credits:** [Nick088](https://linktr.ee/Nick088), Xinntao, Tencent, Geeve George, ai-forever, daroche

In [ ]:
#@title Installation
#@markdown Before running, make sure that you choose
#@markdown * Runtime Type = Python 3
#@markdown * Hardware Accelerator = GPU (Faster, free daily limit of gpu around 12 hours) or CPU (very slow)

#@markdown in the **Runtime** menu -> **Change runtime type**

#@markdown By running this, we clone the repository, set up the envrironment.


from IPython.display import clear_output
import torch

if torch.cuda.is_available():
    device = "cuda"
    print("Using GPU")
else:
    device = "cpu"
    print("Using CPU")

!git clone https://huggingface.co/spaces/Nick088/Real-ESRGAN_Pytorch
%cd Real-ESRGAN_Pytorch
!pip install -r requirements.txt
clear_output()
print("Installed!")

In [ ]:
#@title Run UI

#@markdown The type of tunnel you wanna use for seeing the public link, so that if one of them is down, you can use the other one.
Tunnel = "Gradio" #@param ["Gradio", "Ngrok", "Cloudfare", "LocalTunnel"]

#@markdown Also when using Ngrok ,Cloudfare or LocalTunnel, you need to wait for the Local URL to appear, and only after that click on the Public URL which is above.

#@markdown Use the following option only if you chose Ngrok as the Tunnel:

#@markdown You can get the Ngrok Tunnel Authtoken here: https://dashboard.ngrok.com/tunnels/authtokens/new.

ngrok_tunnel_authtoken = "" #@param {type:"string"}

!sed -i 's/import spaces/ /g'  /content/Real-ESRGAN_Pytorch/app.py
!sed -i 's/@spaces.GPU()/ /g'  /content/Real-ESRGAN_Pytorch/app.py

if Tunnel == "Gradio":
  !sed -i 's/share=False/share=True/g'  /content/Real-ESRGAN_Pytorch/app.py
if Tunnel == "Ngrok":
  !sed -i 's/share=True/share=False/g'  /content/Real-ESRGAN_Pytorch/app.py
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_tunnel_authtoken)
  http_tunnel = ngrok.connect(7860, bind_tls=True)
  print("Ngrok Tunnel Public URL:", http_tunnel.public_url)
elif Tunnel == "Cloudfare":
  !sed -i 's/share=True/share=False/g'  /content/Real-ESRGAN_Pytorch/app.py
  # download cloudfare
  !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb
  !rm -rf nohup.out
  import time
  # Run cloudflare
  !nohup cloudflared tunnel --url localhost:7860 &
  clear_output()
  time.sleep(5)
  # Find and print the Cloudflare URL with a prefix
  cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
  print(f"Cloudfare Tunnel Public URL: {cloudflare_url[0]}")
elif Tunnel == "LocalTunnel":
  !sed -i 's/share=True/share=False/g'  /content/Real-ESRGAN_Pytorch/app.py
  # install
  !npm install -g localtunnel
  import time
  # run localtunnel
  with open('url.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 7860 >> url.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  clear_output()
  print(f"LocalTunnel Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  print(f'LocalTunnel Password: {endpoint_ip}')


!python app.py